In [1]:
import cv2
import os
import csv
import numpy as np
import random
from video_preprocessing import get_params_from_vid, padcrop_image, frame_crop
# from dataset_generation import format_frames

# Folder path where your video clips are located
folder_path = "/Users/rebeccakrall/Desktop/Scratching Clips/not_scratch"
video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4'))]

# File to store labels
label_file = "video_labels.csv"

In [2]:
def generate_random_starts(total_frames, clip_length, fps, total_clips):
    clip_starts = []
    nframes = fps * clip_length
    if (clip_length * fps * total_clips) > total_frames:
        return None

    while len(clip_starts) < total_clips:
        new_start = random.randint(0, int(total_frames - nframes))
        if len(clip_starts) == 0:
            clip_starts.append(new_start)
        elif all([(new_start not in np.arange(x, x+nframes)) and  (new_start + nframes not in np.arange(x, x+nframes)) for x in clip_starts]):
            clip_starts.append(new_start)

    return clip_starts

In [16]:
class LabelGenerator:
    def __init__(self, videos, save_path, clips_per_animal = 10, seconds = 2, buffer = 300, dp = 0.5, minDist = 400, animals = ['m1', 'm2', 'm3', 'm4', 'm5', 'm6']):
        """ Returns a set of frames with their associated label. 

        Args:
            videos: list of filepaths 
            save_path: folder to save new clips
            seconds: length of clip in seconds
        """
        self.videos = videos
        self.save_path = save_path
        self.clips_per_animal = clips_per_animal
        self.seconds = seconds
        self.buffer = buffer
        self.dp = dp
        self.minDist = minDist
        self.animals = animals

        self.current_video = 0
        self.current_animal = 0

    def __iter__(self):

        for vid in self.videos:
            circles, vid_frames, fps, _ = get_params_from_vid(vid, dp = self.dp, minDist = self.minDist)
            for an in self.animals:
                cap = cv2.VideoCapture(vid)
                starts = generate_random_starts(vid_frames, self.seconds, fps, self.clips_per_animal)
                starts.sort()

                filename = os.path.splitext(os.path.basename(vid))[0]
                
                for s in starts:
                    print(s)
                    cap.set(cv2.CAP_PROP_POS_FRAMES, s)
                    print('opened cap')

                    result = []
                    for _ in range(int(self.seconds* fps)):
                      
                        ret, frame = cap.read()
                        center = circles[an]

                        cc, center = frame_crop(center[0], center[1], buffer = self.buffer, shape = frame.shape[:2])
                        pc = padcrop_image(frame, self.buffer, cc, center)
                        result.append(pc)
                            
                    # result = np.array(result)
                    details = {'filename': filename, 'animal': an, 'frames': (s, int(s+(self.seconds*fps))), 'fps': fps, 'shape':frame.shape}
                    yield result, details
        
        cap.release()

In [43]:
videos = ["/Users/rebeccakrall/Data/Video Processing and Prediction/Scratching_Projects/PRX_03/PRX_03_Videos/PRX_03_Run_3_Computer_1_D8_females_15DEC22.mp4"]

video_clips = LabelGenerator(videos= videos, save_path = None, clips_per_animal = 40, seconds = 1.5, animals = ['m2'])

In [44]:
save_path = "/Users/rebeccakrall/Desktop/Behavior Clips"
labels = [x for x in os.listdir(save_path) if '.DS' not in x]

for ind, (v, details) in enumerate(video_clips):
    replay = True
    ind_choice = None
    while replay:
        for frame in v:
            cv2.imshow(f'Clip', frame)
            key = cv2.waitKey(25) & 0xFF
            if key == ord('q'):
                replay = False
            elif key == 32:
                while ind_choice not in [str(i) for i in np.arange(0, len(labels))]:
                    ind_choice = input([f'{ind}: {item}' for ind, item in enumerate(labels)])
                choice = labels[int(ind_choice)]
                print(f'Clip {ind} labeled as {choice}')
                replay = False

    if ind_choice is not None:
        output_video_path = os.path.join(save_path, labels[int(ind_choice)], f"{details['filename']}_{details['animal']}_{details['frames'][0]}_{details['frames'][1]}.mp4")
        print(output_video_path)
        out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), details['fps'], (video_clips.buffer*2, video_clips.buffer*2))
        for frame in v:
            out.write(frame)
        out.release()



    # label = None
    # while label not in potential_behaviors:
    #     label = input(f"Enter label for clip {ind}, \n {[potential_behaviors]}")

# cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


4535
opened cap
Clip 0 labeled as Not Scratch
/Users/rebeccakrall/Desktop/Behavior Clips/Not Scratch/PRX_03_Run_3_Computer_1_D8_females_15DEC22_m2_4535_4579.mp4
6360
opened cap
Clip 1 labeled as Turn Left
/Users/rebeccakrall/Desktop/Behavior Clips/Turn Left/PRX_03_Run_3_Computer_1_D8_females_15DEC22_m2_6360_6404.mp4
9812
opened cap
Clip 2 labeled as Face Grooming
/Users/rebeccakrall/Desktop/Behavior Clips/Face Grooming/PRX_03_Run_3_Computer_1_D8_females_15DEC22_m2_9812_9856.mp4
12968
opened cap
Clip 3 labeled as Body Groom Right
/Users/rebeccakrall/Desktop/Behavior Clips/Body Groom Right/PRX_03_Run_3_Computer_1_D8_females_15DEC22_m2_12968_13012.mp4
14917
opened cap
Clip 4 labeled as Face Grooming
/Users/rebeccakrall/Desktop/Behavior Clips/Face Grooming/PRX_03_Run_3_Computer_1_D8_females_15DEC22_m2_14917_14961.mp4
15425
opened cap
Clip 5 labeled as Not Scratch
/Users/rebeccakrall/Desktop/Behavior Clips/Not Scratch/PRX_03_Run_3_Computer_1_D8_females_15DEC22_m2_15425_15469.mp4
15705
opene

-1

In [43]:


def play_and_label_videos(label_file, video_files):
    # Check if label file exists, if not create it
    if not os.path.exists(label_file):
        with open(label_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Video', 'Label'])
    
    # Load any existing labels
    with open(label_file, 'r') as f:
        reader = csv.reader(f)
        labeled_videos = {rows[0]: rows[1] for rows in reader}
    
    # Iterate over all video files
    for video in video_files:
        if video in labeled_videos:
            print(f"Skipping {video}, already labeled.")
            continue
        
        video_path = os.path.join(folder_path, video)
        cap = cv2.VideoCapture(video_path)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            cv2.imshow('Video', frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

        # Get the label from the user
        label = input(f"Enter label for {video}: ")

        # Save label to file
        with open(label_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([video, label])


In [44]:
play_and_label_videos(label_file, video_files)

Skipping PRX_02_D12_Males_run2_comp1_m13-18clip_2_m1_4829_4848.mp4, already labeled.
Skipping PRX_02_Male_D8_run4_comp1_m37-42clip_1_m5_8468_8673.mp4, already labeled.
Skipping PRX_02_D12_Males_run2_comp1_m13-18clip_2_m3_5146_5216.mp4, already labeled.
Skipping PRX_02_Male_D8_run4_comp1_m37-42clip_1_m4_8278_8403.mp4, already labeled.
Skipping PRX_02_Male_D8_run4_comp1_m37-42clip_1_m1_8630_8673.mp4, already labeled.
Skipping PRX_02_Male_D8_run4_comp1_m37-42clip_3_m4_8407_8627.mp4, already labeled.
Skipping PRX_03_Run_2_Computer_1_D12_Males_16DEC22clip_0_m2_7193_7581.mp4, already labeled.
Skipping PRX_03_Run_2_Computer_1_D12_Males_16DEC22clip_0_m1_8618_8709.mp4, already labeled.
Skipping PRX_03_Run_2_Computer_2_D12_Males_16DEC22clip_0_m1_8813_8831.mp4, already labeled.
Skipping PRX_02_D8_Female_run2_comp2_m67-72clip_5_m3_8879_8913.mp4, already labeled.
Skipping PRX_02_D8_Female_run2_comp2_m67-72clip_5_m5_8524_8595.mp4, already labeled.
Skipping PRX_03_Run_2_Computer_1_D12_Males_16DEC22cl

KeyboardInterrupt: Interrupted by user

In [8]:
# goal - take specific chunks of videos 

Unnamed: 0                                                     0
filepath            PRX-03_Run_3_Computer_1_D8_Males_12DEC22.mp4
mouse id                                                      m3
Video                              D8_Male_Run 3_Comp 1 (m25-30)
Subject #                                                     25
Scorer                                                       Bob
Video Start Time                                               0
Video End Time                                                30
0-5                                                        28.69
10-May                                                     62.83
15-Oct                                                     17.07
15-20                                                      58.06
20-25                                                      73.06
25-30                                                      15.56
Total Time                                                255.27
model_count              

In [4]:
from predict import *
time_dict = {'m1': (0,1), 'm2': (0,1), 'm3': (0,1), 'm4': (0,1), 'm5': (0,1), 'm6': (0,1)}
cg_params = {'bin' : 30, 'n_frames' : 30, 'buffer' : 300, 'output_size': (172,172)}
model_params = {'resolution': 172, 'num_classes': 1, 'input_shape': [8,30,172,172,3], 'checkpoint_path' : "Models/scratch_101524/cp_02.ckpt"}
predict_video(test_file, time_dict, cg_params, model_params, save_dir = save_dir)

In [5]:
test_file = "/Users/rebeccakrall/Data/Video Processing and Prediction/Scratching_Projects/PRX_03/PRX_03_Videos/PRX_03_Run_1_Computer_2_D8_females_15DEC22clip_0.mp4"
save_dir = "/Users/rebeccakrall/Desktop/Test Predict"

In [7]:
predict_video(test_file, time_dict, cg_params, model_params, save_dir = save_dir)

2024-10-16 14:40:35.088620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


True

In [14]:
import matplotlib.pyplot as plt

In [32]:
all_logits = []
with open("/Users/rebeccakrall/Desktop/Test Predict/m6_predict.pkl", 'rb') as f:
    while True:
        try:
            logit = pickle.load(f)
            all_logits.append(float(logit))
        except:
            break
        
fig = plt.figure()
plt.plot(all_logits)
fig.savefig("/Users/rebeccakrall/Desktop/Test Predict/m6_predict.png")

In [34]:
np.where(np.array(all_logits) < 1)

(array([ 8, 24, 42, 58]),)

In [1]:
from predict import *

/Users/rebeccakrall/miniconda3/envs/tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:

test_file = "/Users/rebeccakrall/Data/Video Processing and Prediction/Scratching_Projects/PRX_03/PRX_03_Videos/PRX_03_Run_1_Computer_1_D12_Males_16DEC22clip_2.mp4"
save_dir = "/Users/rebeccakrall/Desktop/Test Predict"
time_dict = {'m1': (0,1), 'm2': (0,1), 'm3': (0,1), 'm4': (0,1), 'm5': (0,1), 'm6': (0,1)}
cg_params = {'bin' : 30, 'n_frames' : 15, 'buffer' : 300, 'output_size': (172,172)}
model_params = {'resolution': 172, 'num_classes': 1, 'input_shape': [8,30,172,172,3], 'checkpoint_path' : "Models/scratch_101724/cp_00.ckpt"}
predict_video_streamwise(test_file, time_dict, cg_params, model_params, save_dir = save_dir)

/Users/rebeccakrall/Code/Multiclass_Behavioral_Classification/video_preprocessing.py:233: RuntimeWarning: Mean of empty slice
  dis = np.nanmean(x_displace[start::3])


True

In [13]:
import matplotlib.pyplot as plt
all_logits = []
with open("/Users/rebeccakrall/Desktop/Test Predict/m2_predict.pkl", 'rb') as f:
    while True:
        try:
            logit = pickle.load(f)
            all_logits.append(float(logit))
        except:
            break
        
fig = plt.figure()
plt.plot(all_logits)
fig.savefig("/Users/rebeccakrall/Desktop/Test Predict/m2_predict.png")